This file is dealing for recovery data
Since recovery data have to be done in different csv files.
This Jupyter file is used to combine all recovery data from different provinces into one

We will joining the following data from these table:
covid19-download.csv: 

**totalcases, numdeaths**

For other 13 table from different provinces: 

**data » total_recoveries**

**data » total_vaccinations**       

**data » total_vaccinated**     

**data » total_boosters_1** 

In [1]:
# !pip install pandas
import pandas as pd

In [8]:
# Read the csv file into a pandas dataframe
# url = 'https://github.com/William-67/CSI4142_Group15/blob/main/COVID19Tracker.ca%20Data%20-%20'
ab = pd.read_csv('COVID19Tracker.ca Data - AB.csv')
bc = pd.read_csv('COVID19Tracker.ca Data - BC.csv')
mb = pd.read_csv('COVID19Tracker.ca Data - MB.csv')
nb = pd.read_csv('COVID19Tracker.ca Data - NB.csv')
nl = pd.read_csv('COVID19Tracker.ca Data - NL.csv')
ns = pd.read_csv('COVID19Tracker.ca Data - NS.csv')
nt = pd.read_csv('COVID19Tracker.ca Data - NT.csv')
nu = pd.read_csv('COVID19Tracker.ca Data - NU.csv')
on = pd.read_csv('COVID19Tracker.ca Data - ON.csv')
pe = pd.read_csv('COVID19Tracker.ca Data - PE.csv')
sk = pd.read_csv('COVID19Tracker.ca Data - SK.csv')
qc = pd.read_csv('COVID19Tracker.ca Data - QC.csv')
yt = pd.read_csv('COVID19Tracker.ca Data - YT.csv')
df = pd.read_csv('covid19-download.csv')

In [5]:
#test 
ab.dtypes

province                              object
last_updated                          object
data » date                           object
data » change_cases                    int64
data » change_fatalities               int64
data » change_tests                    int64
data » change_hospitalizations         int64
data » change_criticals                int64
data » change_recoveries               int64
data » change_vaccinations             int64
data » change_vaccinated               int64
data » change_boosters_1               int64
data » change_boosters_2               int64
data » change_vaccines_distributed     int64
data » total_cases                     int64
data » total_fatalities                int64
data » total_tests                     int64
data » total_hospitalizations          int64
data » total_criticals                 int64
data » total_recoveries                int64
data » total_vaccinations              int64
data » total_vaccinated                int64
data » tot

Only Select recovery data

In [ ]:
origin_list = [ab,bc,mb,nb,nl,ns,nt,nu,on,pe,sk,qc,yt]
modify_list = []
for province in origin_list:
    temp = province[["data » date",,"data » total_recoveries","data » total_vaccinations","data » total_vaccinated","data » total_boosters_1"]]
    temp = temp.rename({'data » date':'date','data » total_recoveries': 'recovery'})
    temp = temp.rename({'data » total_vaccinations':'partial'})
    temp = temp.rename({'data » total_vaccinated':'full'})
    temp = temp.rename({'data » total_boosters_1':'booster'})
    recovery_list.append(temp)
recovery_
